<div style="float: right; margin: 20px 20px 20px 20px"><img src="images/bro.png" width="100px"></div>

# Replicating ZQ/ZQL Demo
This script replicates much of the ZQ functionality shown in this 'Zeek at Home' Demo (from Brim Security):
- https://www.youtube.com/watch?v=ldrEadAQYTM. 

In this notebook we'll load up some Zeek logs, convert the logs to Parquet files. We also perform some data exploration, querying, and analysis with Spark. 

### Context
Brim Security makes a cool product, in addition, they are open sourcing all of the code shown in their youtube 'Zeek at Home' demo. So two big thumbs up for that. Having new functionality and alternatives is always a good thing, so please go check out their product and give them a 'star' on their repos. 

This notebook is sharing a personal perspective, in particular I'm a big fan of using/leveraging popular toolkits/libraries whenever possible. One of the goals of my software is to help 'bridge' from one cool thing to another. For instance **Zeek to Parquet** or **Zeek to Spark** or **Zeek to Kafka to Spark**. When you're leveraging popular open source your often taking advantage of **>10k person hours of work**, so NOT doing this should be viewed with a large amount of scepticism. :)

<div style="float: right; margin: 30px 0px 0px 0px"><img src="images/parquet.png" width="280px"></div>

### Opinions
Obviously opinions are subjective and the 'right thing' is based on your use cases.. but from my experience using popular open source libraries/formats like Spark and Parquet give you significant benefits for **most** use cases. In particular both Spark and Parquet 'scale up' from laptop, to beefy server, to AWS/Azure mega clusters. So from a developmentment tool chain perspective, you can prototype on your laptop and then use the same code/approaches as you scale up.
- **Parquet** > ZNG
- **Spark** >> ZQ/ZQL
<br><br><br>

<div style="float: right; margin: -80px 0px 0px 0px"><img src="images/spark.png" width="220px"></div>

### Notebook Install Requirements
```pip install zat```

### Notes
- Zeek Analysis Tools (ZAT) was shown at BroCon 2017: https://youtu.be/pG5lU9CLnIU
- ZAT is simply a 'helper' library, it helps you convert Zeek logs to Parquet and makes loading Zeek data into Spark easy.
- In addition to the tiny bit of functionality shown here, Spark has awesome data exploration, transformations, pipelines, and of course, machine learning libraries.
  - https://nbviewer.jupyter.org/github/SuperCowPowers/zat/blob/master/notebooks


### Software
- Zeek Analysis Tools (ZAT): https://github.com/SuperCowPowers/zat
- Parquet: https://parquet.apache.org
- Spark: https://spark.apache.org

### Data
- This script uses a Zeek CONN log with ~23 million rows and also a HTTP log
- Both datasets are available here: https://data.kitware.com/#collection/58d564478d777f0aef5d893a

In [1]:
# Third Party Imports
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# ZAT imports
import zat
from zat import log_to_sparkdf

# Good to print out versions of stuff
print('ZAT: {:s}'.format(zat.__version__))
print('PySpark: {:s}'.format(pyspark.__version__))

ZAT: 0.3.9
PySpark: 3.0.0


<div style="float: right; margin: 20px 20px 20px 20px"><img src="images/spark.png" width="200px"></div>

# Spark It!
### Spin up Spark with 4 Parallel Executors
Here we're spinning up a local spark server with 4 parallel executors, although this might seem a bit silly since we're probably running this on a laptop, there are a couple of important observations:

<div style="float: right; margin: 20px 20px 20px 20px"><img src="images/spark_jobs.png" width="400px"></div>

- If you have 4/8 cores use them!
- It's the exact same code logic as if we were running on a distributed cluster.
- We run the same code on **DataBricks** (www.databricks.com) which is awesome BTW.



In [2]:
# Spin up a local Spark Session (with 4 executors)
spark = SparkSession.builder.master('local[4]').appName('my_awesome').getOrCreate()

In [5]:
# Use the ZAT class to load our log file into a Spark dataframe
spark_it = log_to_sparkdf.LogToSparkDF(spark)

# Read in Zeek HTTP Log
data_path = '/Users/briford/data/bro/http.log'  # Obviously you'll need to change this :)
http_df = spark_it.create_dataframe(data_path)

<div style="float: right; margin: 0px 0px 0px -80px"><img src="images/spark_distributed.png" width="500px"></div>

# Spark Workers and Data Partitions
Spark will read in and partition the data out to our workers. Our dataframe(rdd) will have some number of partitions that are divided up amongst the worker pool. Each worker will operate on only a subset of the data and Spark will manage the 'magic' for how that work gets run, aggregated and presented.


**Image Credit:** Jacek Laskowski, please see his excellent book - Mastering Apache Spark  https://jaceklaskowski.gitbooks.io/mastering-apache-spark

In [7]:
http_df.rdd.getNumPartitions()

11

## Replicate functionality shown at https://youtu.be/ldrEadAQYTM?t=923
We're using HTTP data instead of SMB but this basically replicates the group/aggregate/count functionality they showed in the demo.

In [8]:
http_df.groupby('method', 'status_code').count().sort('count', ascending=False).show()

+-------+-----------+-------+
| method|status_code|  count|
+-------+-----------+-------+
|   HEAD|        404|1294022|
|    GET|        404| 429283|
|   POST|        200| 125638|
|    GET|        200|  88631|
|   POST|          0|  32918|
|    GET|        400|  29152|
|    GET|        303|  10858|
|    GET|        403|   8530|
|   POST|        404|   4277|
|    GET|        304|   3851|
|    GET|        302|   3250|
|    GET|          0|   2906|
|    GET|        401|   2159|
|OPTIONS|        200|   1897|
|   POST|        302|   1226|
|   HEAD|        503|   1010|
|   POST|        206|    869|
|    GET|        301|    642|
|   HEAD|          0|    606|
|    GET|        503|    550|
+-------+-----------+-------+
only showing top 20 rows



<div style="float: right; margin: 30px 0px 0px 0px"><img src="images/parquet.png" width="400px"></div>

# Convert my Zeek logs to Parquet files
Apache Parquet is a columnar storage format focused on performance. Here's we going to convert our Zeek/Zeek log to a Parquet file is one line of code. The conversion is super scalable since we're using spark distributed executors to do the conversions.

In [9]:
# Read in a Zeek CONN Log
data_path = '/Users/briford/data/bro/conn.log'  # Obviously you'll need to change this :)
conn_df = spark_it.create_dataframe(data_path)

In [10]:
# DataFrames can be saved as Parquet files, maintaining the schema information.
conn_df.write.parquet('conn.parquet', compression='gzip')

In [11]:
# Have Spark read in the Parquet File
conn_df = spark.read.parquet('conn.parquet')

<div style="float: right; margin: 20px 20px 20px 20px"><img src="images/compressed.jpeg" width="300px"></div>

# Parquet files are compressed
Here we see the first benefit of Parquet which stores data with compressed columnar format. There are several compression options available (including uncompressed).

## Original conn.log = 2.5 GB 
## conn.parquet = ~420MB

<div style="float: left; margin: 20px 20px 20px 20px"><img src="images/nuked_crop.jpg" width="150px"></div>


# Light it Up!
Now that we have our Parquet data loaded into Spark, we're going to demonstrate just a few simple Spark operations but obviously you now have the full power of the Death Star in your hands.

<div style="float: left; margin: 20px 0px 0px 50px"><img src="images/spark_sql.jpg" width="150px"></div>
<div style="float: left; margin: 0px 0px 0px 50px"><img src="images/mllib.png" width="150px"></div>

In [12]:
# Get information about the Spark DataFrame
num_rows = conn_df.count()
print("Number of Rows: {:d}".format(num_rows))
columns = conn_df.columns
print("Columns: {:s}".format(','.join(columns)))

Number of Rows: 22694356
Columns: ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents


<div style="float: right; margin: 20px 20px 20px 20px"><img src="images/fast.jpg" width="350px"></div>

# Did we mention fast?
The query below was executed on 4 workers. The data contains over 22 million Zeek conn log entries and the time to complete was a **fraction of a second** running on my Mac Laptop :)

In [16]:
# Lets look at some 'service' breakdowns in our Zeek conn log
conn_df = conn_df.filter(conn_df['service'] != '-')
%timeit -r 1 -n 1 conn_df.groupby('proto','service').count().sort('count', ascending=False).show()   

+-----+--------+------+
|proto| service| count|
+-----+--------+------+
|  tcp|    http|445214|
|  udp|     dns|160559|
|  tcp|     ssl| 49017|
|  tcp|     ssh|  4778|
|  udp|    dhcp|  3052|
|  tcp|ftp-data|  2880|
|  tcp|     ftp|  2675|
|  tcp|     dns|   706|
|  tcp|    smtp|   194|
|  tcp|    pop3|     2|
+-----+--------+------+

522 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Replicate 'power user' functionality shown at https://youtu.be/ldrEadAQYTM?t=941
We'll do a 'pipeline' of operations similar to the demo
- First select just the columns we want (CUT)
- Next create a new column with an aggregation of other columns
- Last do the avg and sum of the total bytes

In [17]:
# First select just the columns we want (CUT)
cut_df = conn_df.select(['uid', 'orig_bytes', 'resp_bytes'])

# Next create a new column with an aggregation of other columns
cut_df = cut_df.withColumn('total_bytes', sum([cut_df['orig_bytes'], cut_df['resp_bytes']]))
no_zero_df = cut_df.filter(cut_df['total_bytes'] != 0)
no_zero_df.show(5)

# Last do the avg and sum of the total bytes
cut_df.select(F.avg(F.col('total_bytes')), F.sum(F.col('total_bytes'))).show()

+------------------+----------+----------+-----------+
|               uid|orig_bytes|resp_bytes|total_bytes|
+------------------+----------+----------+-----------+
| C7bMbs9ZbZOfKk2o7|     29737|     15395|      45132|
| CJcxhQH5ac9BFaA05|        62|         0|         62|
|CaikKr23aO1enqyq6j|        48|         0|         48|
|CPcix93BFygB2ANPtj|     29515|     15395|      44910|
|C0plQI2cQSN3eC2ROh|     29597|     15395|      44992|
+------------------+----------+----------+-----------+
only showing top 5 rows

+-----------------+----------------+
| avg(total_bytes)|sum(total_bytes)|
+-----------------+----------------+
|68829.53709214335|     46052260189|
+-----------------+----------------+



# Okay now perhaps something a bit more useful
- First filter out unknown services
- More meaningful aggregation based on IP, port, and service

In [19]:
# First filter out unknown services
conn_df = conn_df.filter(conn_df['service'] != '-')

# More meaningful aggregation based on IP, port, and service
conn_df.groupby('id_orig_h', 'id_resp_p', 'service') \
    .agg(F.sum('orig_bytes').alias('total_bytes')) \
    .sort('total_bytes', ascending=False).show()

+---------------+---------+-------+-----------+
|      id_orig_h|id_resp_p|service|total_bytes|
+---------------+---------+-------+-----------+
| 192.168.202.88|       22|    ssh|  491259422|
| 192.168.203.63|       80|   http|  381506783|
|192.168.202.102|       80|   http|   80956460|
| 192.168.202.79|       80|   http|   54699732|
| 192.168.202.81|       22|    ssh|   42247132|
|192.168.202.110|       80|   http|   37952120|
|192.168.202.118|       80|   http|   18731116|
|192.168.202.110|      443|    ssl|   17883212|
| 192.168.202.95|       22|    ssh|   13947240|
| 192.168.202.96|       80|   http|   11871726|
|192.168.202.138|       80|   http|   10689231|
| 192.168.202.65|      443|    ssl|    8550078|
|192.168.202.140|       80|   http|    7860895|
| 192.168.204.45|    55553|    ssl|    6489031|
|192.168.202.110|     8080|   http|    5595350|
|192.168.202.140|      443|    ssl|    4883939|
|192.168.202.125|       80|   http|    4289446|
|192.168.202.141|       80|   http|    4

<div style="float: right; margin: 50px 0px 0px 20px"><img src="images/deep_dive.jpeg" width="350px"></div>

# Data looks good, lets take a deeper dive
Spark has a powerful SQL engine as well as a Machine Learning library. So now that we've got the data loaded into a Spark Dataframe we're going to utilize Spark SQL commands to do some investigation and clustering using the Spark MLLib. For this deeper dive we're going to go to another notebook :)

### Spark Clustering Notebook
- [Zeek Spark Clustering](https://nbviewer.jupyter.org/github/SuperCowPowers/zat/blob/master/notebooks/Spark_Clustering.ipynb)

<div style="float: left; margin: 0px 0px 0px 0px"><img src="images/spark_sql.jpg" width="150px"></div>
<div style="float: left; margin: -20px 50px 0px 0px"><img src="images/mllib.png" width="150px"></div>